In [ ]:
#UKBB Analysis - Insulin Resistance and CKD

#library#
library(utils)
library(psych)
library(rms)
library(naniar)
library(tidyverse)
library(dplyr)
library(table1)
library(readr)
library(stringr)
library(R.utils)
library(data.table)
library(rlang)

In [ ]:
#Pull dataset from this path location
path <- "/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/"

#file directory
print(list.files(path))
#Data import
UKBB_AG_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/UKBB_AG.txt", header = TRUE, na.strings=c("",".","NA"))
UKBB_AG=as.data.frame(UKBB_AG_m)
rm(UKBB_AG_m)

UKBB_AG2_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/AG_VARS2.txt", header = TRUE, na.strings=c("",".","NA"))
UKBB_AG2=as.data.frame(UKBB_AG2_m)
rm(UKBB_AG2_m)

DBCMED_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/DBCMED.txt", header = TRUE, na.strings=c("",".","NA"))
DBCMED=as.data.frame(DBCMED_m)
rm(DBCMED_m)

MED_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/MED.txt", header = TRUE, na.strings=c("",".","NA"))
MED=as.data.frame(MED_m)
rm(MED_m)

SDXILL_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/SDXILL.txt", header = TRUE, na.strings=c("",".","NA"))
SDXILL=as.data.frame(SDXILL_m)
rm(SDXILL_m)

SDXOP_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/SDXOP.txt", header = TRUE, na.strings=c("",".","NA"))
SDXOP=as.data.frame(SDXOP_m)
rm(SDXOP_m)

UKBINP_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/UKBINP.txt", header = TRUE, na.strings=c("",".","NA"))
UKBINP=as.data.frame(UKBINP_m)
rm(UKBINP_m)

UKBOP_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/UKBOP.txt", header = TRUE, na.strings=c("",".","NA"))
UKBOP=as.data.frame(UKBOP_m)
rm(UKBOP_m)

#WHR ratio
WHR_m <- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/WHR_VARS.txt", header = TRUE, na.strings=c("",".","NA"))
WHR=as.data.frame(WHR_m)

#import PGS files
#LIPID-mediated Insulin Resistance score
PGS_IR <- read.table("~/jupyter/UKBB_IR_GRS.txt", header = TRUE, na.strings=c("",".","NA"))
PGS_IR <- PGS_IR[,c('FID','IID','GRS_RAW','GRS_WT')]
#Insulin resistance score
PGS_2IR <- read.table("~/jupyter/IRKD_SNP/UKBB_IRLOTTA_GRS_09Sep21.txt", header = TRUE, na.strings=c("",".","NA"))
PGS_2IR <- PGS_2IR[,c('FID','IID','GRS_RAW','GRS_WT')]
#53SNP Lotta Insulin Resistance Score
PGS_53IR <- read.table("~/jupyter/IRKD_SNP/UKBB_IRLOTTA53SNPs_GRS_09SEP21.txt", header = TRUE, na.strings=c("",".","NA"))
PGS_53IR <- PGS_53IR[,c('FID','IID','GRS_RAW','GRS_WT')]
#Type 2 diabetes score (revised as of 05.2021 to replace with score containing no UKBB overlap)
T2D_2IR <- read.table("~/jupyter/IRKD_SNP/UKBB_T2D_MVMR_GRS_REV_20OCT21.txt", header = TRUE, na.strings=c("",".","NA"))
T2D_2IR <- T2D_2IR[,c('FID','IID','GRS_RAW','GRS_WT')]
#DM1_PROB5 - T1D probable, DM_POS3 - T2D Eastwood questionaires, DM_POS7 - T2D Alg and labs
T2D <- read.table("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/UKB_T2DM.txt", header = TRUE, na.strings=c("",".","NA"))
#Langour (two levels of significance)
Lagou_5e8 <- read.table("/cellar/users/agarduno/jupyter/Lagou_2019_5e8_28Nov20.txt", header = TRUE, na.strings=c("",".","NA"))
Lagou_1e5 <- read.table("/cellar/users/agarduno/jupyter/Lagou_2019_1e5_28Nov20.txt", header = TRUE, na.strings=c("",".","NA"))
#List of Individuals - Unrelated through PCA
PCAunrelated_m <- fread("/cellar/users/agarduno/jupyter/IRKD_SNP/Alexis_T2D_Unrelated_EUR_PCA_Final.txt", sep=" ",header = TRUE, na.strings=c("",".","NA"))
PCAunrelated=as.data.frame(PCAunrelated_m)
rm(PCAunrelated_m)
PCAunrelated <- PCAunrelated %>% select(FID_Salem,T2D_status,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
#names(PCAunrelated)
#head(PCAunrelated)
#First occurrences
UKBFIRST<- fread("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/UKBFIRST.txt", header = TRUE, na.strings=c("",".","NA"))
#further PGS renaming
names(PGS_IR)[names(PGS_IR)=="GRS_RAW"] <- "GRS_RAW_LIR"
names(PGS_IR)[names(PGS_IR)=="GRS_WT"] <- "GRS_WT_LIR"
names(PGS_2IR)[names(PGS_2IR)=="GRS_RAW"] <- "GRS_RAW_IR"
names(PGS_2IR)[names(PGS_2IR)=="GRS_WT"] <- "GRS_WT_IR"
names(PGS_53IR)[names(PGS_53IR)=="GRS_RAW"] <- "GRS_RAW_IR53"
names(PGS_53IR)[names(PGS_53IR)=="GRS_WT"] <- "GRS_WT_IR53"
names(T2D_2IR)[names(T2D_2IR)=="GRS_RAW"] <- "GRS_RAW_T2DIR"
names(T2D_2IR)[names(T2D_2IR)=="GRS_WT"] <- "GRS_WT_T2DIR"
names(Lagou_5e8)[names(Lagou_5e8)=="GRS_RAW"] <- "GRS_RAW_L5E8IR"
names(Lagou_5e8)[names(Lagou_5e8)=="GRS_WT"] <- "GRS_WT_L5E8IR"
names(Lagou_1e5)[names(Lagou_1e5)=="GRS_RAW"] <- "GRS_RAW_L1E5IR"
names(Lagou_1e5)[names(Lagou_1e5)=="GRS_WT"] <- "GRS_WT_L1E5IR"

In [ ]:
#names(UKBB_AG)
#names(UKBB_AG2)
#names(DBCMED)
#names(MED)
#names(SDXILL)
#names(SDXOP)
#names(UKBINP)
#names(UKBOP)
#names(PGS_IR)
#names(PGS_2IR)
#names(PGS_53IR)
#names(T2D)
#names(UKBFIRST)
#names(WHR)

In [ ]:
#Sample Restriction
paste0('Original')
dim(UKBB_AG)

#Merge the PGS files
paste0('Genetic Risk Score')
dim(PGS_IR)
dim(PGS_2IR)
PGS <- merge(PGS_IR,PGS_2IR,by="FID") #insulin resistance
PGS <- merge(PGS,PGS_53IR,by="FID") #insulin resistance (full Lotta score)
PGS <- merge(PGS,T2D_2IR,by="FID") #T2D
PGS <- merge(PGS,Lagou_5e8,by="FID") #Langour, weighted IR PGS (v1)
PGS <- merge(PGS,Lagou_1e5,by="FID") #Langour, weighted IR PGS (v2)
paste0('Genetic Risk Score Combo')
dim(PGS)
#Add the linker file for the PGS
linker <- read.table('~/jupyter/UKBiobank_genoQC_allancestry_linker.txt', header = TRUE, na.strings=c("",".","NA"))
#Merge linker to PGS
PGS <- merge(PGS,linker,by="FID")
names(PGS)[names(PGS)=="FID_Salem"] <- "f.eid" #magic!
#Restriction 1: PGS score available
PGS <- merge(PGS,WHR,by='f.eid',all.x=TRUE) 
paste0('People with WHR')
dim(UKBB_AG2)
UKBB_AG2 <- merge(UKBB_AG,PGS,by='f.eid') 
paste0('People with PGS')
dim(UKBB_AG2)
#UKBB_AG2 <- merge(PGS_2IR,UKBB_AG2,by='f.eid',all.x=TRUE)
#dim(UKBB_AG2)

#Restriction 2: European-only analysis --> (Developed by: Erin Richard)
UKBB_AG <- UKBB_AG2 %>%
mutate(RACE=ifelse(ETHNICITY.0.0=="British" & !is.na(ETHNICITY.0.0) , "W",
            ifelse(ETHNICITY.0.0=="Irish" & !is.na(ETHNICITY.0.0),"W",  
            ifelse(ETHNICITY.0.0=="White" & !is.na(ETHNICITY.0.0),"W",  
            ifelse(ETHNICITY.0.0=="Any other white background" & !is.na(ETHNICITY.0.0),"W",
            ifelse(ETHNICITY.0.0=="Caribbean" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="African" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="Any other Black background" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="Black" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="White and Black Caribbean" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="Black or Black British" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="White and Black African" & !is.na(ETHNICITY.0.0),"B",
            ifelse(ETHNICITY.0.0=="Asian or Asian British" & !is.na(ETHNICITY.0.0),"A",
            ifelse(ETHNICITY.0.0=="Any other Asian background" & !is.na(ETHNICITY.0.0),"A",
            ifelse(ETHNICITY.0.0=="Indian" & !is.na(ETHNICITY.0.0),"A",
            ifelse(ETHNICITY.0.0=="Bangladeshi" & !is.na(ETHNICITY.0.0),"A",
            ifelse(ETHNICITY.0.0=="Pakistani" & !is.na(ETHNICITY.0.0),"A",
            ifelse(ETHNICITY.0.0=="Chinese" & !is.na(ETHNICITY.0.0),"A",              
            ifelse (is.na(ETHNICITY.0.0), NA,"O")))))))))))))))))))%>%
mutate(SASIAN=case_when(ETHNICITY.0.0=="Bangladeshi"|ETHNICITY.0.0=="Indian"|ETHNICITY.0.0=="Pakistani"~1, ))%>%
mutate(CARIBN=case_when(ETHNICITY.0.0=="Caribbean"~1, ))

UKBB_AG <- UKBB_AG %>% filter(RACE=="W")
paste0('European people')
dim(UKBB_AG)
#check PRS
#names(UKBB_AG2)

In [ ]:
#names(PGS_2IR)
#names(PGS_53IR)
#names(PGS)

In [ ]:
##################KIDNEY DISEASE PHENOTYPES##################
#Section Description: inspect existing renal variables,EGFR, QA of EGFR, 12 Phenotypes

##################Existing Renal Variables##################
#Inspect distributions of input variables for kidney disease
dim(UKBB_AG)
describe(UKBB_AG$'URN_ALBUMIN.0.0') #units (mg/L), https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30500
print('URN Flag')
table(UKBB_AG$'URN_ALBUMIN_FLAG.0.0')
describe(UKBB_AG$'URN_CREATININE.0.0') #original showcase (umol/L), https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30700

#CKD-EPI (eGFR)
describe(UKBB_AG$'ETHNICITY.0.0')
table(UKBB_AG$'ETHNICITY.0.0')
describe(UKBB_AG$'AGE_BASE.0.0')
summary(UKBB_AG$'URN_CREATININE.0.0')
summary(UKBB_AG$'SEX.0.0')
##################EGFR variable##################

#tree: (i) black/white, (ii) sex, (iii) serum creatinine
#egfr
UKBB_AG$CREATININE_SRM2.0.0 <- UKBB_AG$CREATININE_SRM.0.0*0.0113
UKBB_AG$EGFR.0.0 <- ifelse(UKBB_AG$'RACE'!='W'
               
                   #black
                   ,ifelse(UKBB_AG$'SEX.0.0' == 'Female' #black female
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=62
                             ,166*((UKBB_AG$'CREATININE_SRM2.0.0'/0.7)^(-0.329))*((0.993)^(UKBB_AG$'AGE_BASE.0.0'))
                             ,166*((UKBB_AG$'CREATININE_SRM2.0.0'/0.7)^(-1.209))*((0.993)^UKBB_AG$'AGE_BASE.0.0'))                                
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=80
                              ,163*((UKBB_AG$'CREATININE_SRM2.0.0'/0.9)^(-0.411))*((0.993)^UKBB_AG$'AGE_BASE.0.0')
                              ,163*((UKBB_AG$'CREATININE_SRM2.0.0'/0.9)^(-1.209))*((0.993)^UKBB_AG$'AGE_BASE.0.0')
                              )
                    ) #black, male
                   
                    #white or other
                    ,ifelse(UKBB_AG$'SEX.0.0' == 'Female' #white female
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=62
                             ,144*((UKBB_AG$'CREATININE_SRM2.0.0'/0.7)^(-0.329))*((0.993)^UKBB_AG$'AGE_BASE.0.0')
                             ,144*((UKBB_AG$'CREATININE_SRM2.0.0'/0.7)^(-1.209))*((0.993)^UKBB_AG$'AGE_BASE.0.0'))                                
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=80
                              ,141*((UKBB_AG$'CREATININE_SRM2.0.0'/0.9)^(-0.411))*((0.993)^UKBB_AG$'AGE_BASE.0.0')
                              ,141*((UKBB_AG$'CREATININE_SRM2.0.0'/0.9)^(-1.209))*((0.993)^UKBB_AG$'AGE_BASE.0.0')
                              )
                    ))

#estimated EGFR flag (factor)   
#code before restriction (to only white)
UKBB_AG$EGFR_flag.0.0 <- ifelse(UKBB_AG$'RACE'!='W'
               
                   #black
                   ,ifelse(UKBB_AG$'SEX.0.0' == 'Female' #black female
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=62
                             ,1
                             ,2)                                
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=80
                              ,3
                              ,4)
                              
                    ) #black, male
                   
                    #white or other
                    ,ifelse(UKBB_AG$'SEX.0.0' == 'Female' #white female
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=62
                             ,5
                             ,6)                                
                      ,ifelse(UKBB_AG$'CREATININE_SRM.0.0'<=80
                              ,7
                              ,8)
                              
                    )) %>% factor(labels=c("whitefemalelow","whitefemalehigh","whitemalelow","whitemalehigh"))

#qa - cross-check groups
table(UKBB_AG$EGFR_flag.0.0,UKBB_AG$'SEX.0.0')
table(UKBB_AG$EGFR_flag.0.0,UKBB_AG$'ETHNICITY.0.0')
summary(UKBB_AG$'EGFR_flag.0.0') #33,147 NAs (gender or race missing)
summary(UKBB_AG$'EGFR.0.0') #mean - 9.56

##################Generate Kidney Disease Phenotypes##################

##Phenotype 1##
#Albumineria (albumin-creatinine ratio (mg/g))

#note, albumin - is in mg/L.
#note, CREATININE_SRM2.0.0 is in mg/dl. 
#ACR (mg/g) - 0.1*albumin (mg/L) --> albumin (mg/dl)/ (URINE_SRM2.0.0 (mg/dl) --> 0.001*g/dl)
#not sure about why ACR seems right if I don't transform the variables
#convert umol/l to g/dl (correct units)
UKBB_AG$'URN_CREATININE2.0.0' <- UKBB_AG$'URN_CREATININE.0.0'*(1/88.4)
UKBB_AG$'ACR.0.0' <- (0.1*UKBB_AG$'URN_ALBUMIN.0.0')/(0.001*UKBB_AG$'URN_CREATININE2.0.0') #urine creatinine (check units)
#level of detection --> don't have that many missing for urine creatinine
#urine albumin flag --> URN_ALBUMINFLAG.0.0 (<6.7) --> 

#albumineria
UKBB_AG$'ALBUMINERIA.0.0'  <- ifelse(UKBB_AG$'ACR.0.0' <30,
                                  "normo",
                                  ifelse(UKBB_AG$'ACR.0.0' >= 30 & UKBB_AG$'ACR.0.0' < 300
                                         ,"micro"
                                         ,ifelse(UKBB_AG$'ACR.0.0' >= 30 & UKBB_AG$'ACR.0.0' >= 300,
                                                 "macro",NA)))

pmiss <- UKBB_AG %>% summarise_each(funs(100*mean(is.na(.)))) %>% gather() %>% arrange(desc(value))

#below levels of detection --> normo
UKBB_AG <- UKBB_AG %>% mutate(ALBUMINERIA.0.0=case_when(URN_ALBUMIN_FLAG.0.0 =='<6.7'~"normo",TRUE~ALBUMINERIA.0.0)) %>%
             mutate(ALBUMINERIA.0.0=case_when(URN_CREATININE_FLAG.0.0 =='<88'~"normo",TRUE~ALBUMINERIA.0.0)) 

pmiss2 <- UKBB_AG %>% summarise_each(funs(100*mean(is.na(.)))) %>% gather() %>% arrange(desc(value))

#qa check albumineria by cross-tab with ACR
#head((UKBB_AG$'URN_ALBUMIN.0.0'))
#head(UKBB_AG$'URN_CREATININE2.0.0')
#head(UKBB_AG$'ACR.0.0')
aggregate(ACR.0.0~ALBUMINERIA.0.0, data=UKBB_AG, mean, na.rm=TRUE)
aggregate(ACR.0.0~ALBUMINERIA.0.0, data=UKBB_AG, min, na.rm=TRUE)
aggregate(ACR.0.0~ALBUMINERIA.0.0, data=UKBB_AG, max, na.rm=TRUE)
summary(UKBB_AG$ACR.0.0)

##Phenotype 2 (ESKD)##
#ESKD - any source of ESKD from algorithm
UKBB_AG$'ESKD.0.0' <- ifelse(is.na(UKBB_AG$ESRDSOURCE)==FALSE | UKBB_AG$'EGFR.0.0' < 15,"yes","no") %>% factor()

##Phenotype 3 (CKD)##
UKBB_AG$'CKD.0.0' <- ifelse(UKBB_AG$EGFR.0.0 >= 60,
                                  "CKD controls",
                                  ifelse(UKBB_AG$EGFR.0.0 < 60 & UKBB_AG$EGFR.0.0 >= 45
                                         ,"CKD"
                                         ,ifelse(UKBB_AG$EGFR.0.0,
                                                 "CKD extreme",NA))) %>% factor()
                                       
#qa of CKD
aggregate(EGFR.0.0~CKD.0.0, data=UKBB_AG, mean, na.rm=TRUE)
aggregate(EGFR.0.0~CKD.0.0, data=UKBB_AG, min, na.rm=TRUE)
aggregate(EGFR.0.0~CKD.0.0, data=UKBB_AG, max, na.rm=TRUE)
summary(UKBB_AG$EGFR.0.0)  

##Phenotype 4 (DN)##
#DN - macro or ESKD 
UKBB_AG$'DN.0.0' <- ifelse(UKBB_AG$'ESKD.0.0'=="yes" | UKBB_AG$'ALBUMINERIA.0.0'=="macro",
                           "yes","no") %>% factor()

##Phenotype 5 (All conditions)##
#All conditions including micro or macro or ESKD
UKBB_AG$'ALL.0.0' <- ifelse(UKBB_AG$'ESKD.0.0'=="yes" | UKBB_AG$'ALBUMINERIA.0.0'=="macro"
                           | UKBB_AG$'ALBUMINERIA.0.0'=="micro",
                           "yes","no") %>% factor()

##Phenotype 6 (Non-ESKD)##
#All conditions including micro or macro or ESKD
UKBB_AG$'NONESKD.0.0' <- ifelse(UKBB_AG$'ALBUMINERIA.0.0'=="normo" | UKBB_AG$'ALBUMINERIA.0.0'=="macro"
                           | UKBB_AG$'ALBUMINERIA.0.0'=="micro",
                           "yes","no") %>% factor()

##Phenotype 7 (DN-CKD)##
UKBB_AG$'DNCKD.0.0' <- ifelse((UKBB_AG$EGFR.0.0 <45 & UKBB_AG$'ALBUMINERIA.0.0' %in% c("micro","macro"))
                              | UKBB_AG$'ESKD.0.0'=="yes","yes","no") %>% factor()
UKBB_AG$'CTRL_DNCKD.0.0' <- ifelse((UKBB_AG$EGFR.0.0 >= 60 & UKBB_AG$'ALBUMINERIA.0.0'=="normo"),
                           "yes","no") %>% factor()

##summary statistics##
xtabs(~ALBUMINERIA.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~ESKD.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~CKD.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~DN.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~ALL.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~NONESKD.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~DNCKD.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
xtabs(~CTRL_DNCKD.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)
summary(UKBB_AG$ACR.0.0)
summary(UKBB_AG$EGFR.0.0)

#review albumin missing
write.table(pmiss, file="~/jupyter/Analysis/percentmiss_12Jan21_preflag.txt", quote=FALSE, sep="\t", row.names=FALSE)
write.table(pmiss2, file="~/jupyter/Analysis/percentmiss_12Jan21_postflag.txt", quote=FALSE, sep="\t", row.names=FALSE)



In [ ]:
#Type 2 Diabetics
#Description: Erin has generated T2D diabetic cases using UKBB_COMP_1YR
#specifically questionaire and first occurrence data

#First Occurrences
####################NEED TO LIMIT DATE#######################
#merge type 2 cases previously identified
UKBB_AG <- merge(UKBB_AG,T2D, by="f.eid", all=FALSE)
#merge with first occurrence data
BASEDATE<-UKBB_AG %>% select(f.eid, VISDATE.0.0,DM1_PROB5,DM_POS3,DM_POS7)
UKBFIRSTbase<-merge(UKBFIRST, BASEDATE, by="f.eid", all=FALSE)
#head(UKBFIRSTbase)
UKBFIRSTbase$E10DATE<-as.Date(UKBFIRSTbase$E10DATE.0.0)
UKBFIRSTbase$E11DATE<-as.Date(UKBFIRSTbase$E11DATE.0.0)
UKBFIRSTbase$VISDATE<-as.Date(UKBFIRSTbase$VISDATE.0.0)
UKBFIRSTbase$VISDATE_1Y<-as.Date(UKBFIRSTbase$VISDATE.0.0)+365.25
#keep if FO precedes one year after baseline
UKBFIRST1Y<-UKBFIRSTbase  %>% filter(E11DATE<=VISDATE_1Y) #only keep T2D

#E10 - T1D FO, E11 - T2D FO
#mutate - generate new variable; case when - same as sql case when
#allows multiple uses of if-else statements
#pipe passes dataset, create multiple variables at once
UKBFIRST2 <- UKBFIRST1Y %>%
#generate flag if not na - already filtered
mutate(E11_FO= case_when(!is.na(E11DATE) ~1,))
head (UKBFIRST2, n=10)
xtabs(~E11_FO, UKBFIRST2, exclude=NULL, na.action=na.pass)
UKBFIRST2 <- UKBFIRST2 %>% select(f.eid, E11_FO)

#T2D Diabetics - add E11 first occurrences
#multiple case when -> more specific to more general
#T1D = -9, T2D=1, T1D=0
UKBB_AG <- merge(UKBB_AG,UKBFIRST2, by="f.eid", all=TRUE)
UKBB_AG <- UKBB_AG %>% mutate(T2D.0.0=case_when(DM1_PROB5==1 ~ -9,#T1D
                                          (E11_FO==1 | DM_POS7==1) ~ 1, #T2D
                                           TRUE ~ 0))

xtabs(~T2D.0.0, UKBB_AG, exclude=NULL, na.action=na.pass)

#OLD DIABETES ALGORITHM TEMPLATE - 10.29.20
#Diabetes Algorithm (temporary diabetes)
# subset <- MED %>% select('f.eid', 'TRTMEDCODE.0.0')
# UKBB_AG <- merge(UKBB_AG, subset, by='f.eid', all.x=T)

#review diabetes variables
# describe(UKBB_AG$DIAB_SDX.0.0)
#26,400, only have field from baseline
# describe(UKBB_AG$DIAB_SDX_INS.0.0)
# table(UKBB_AG$DIAB_SDX.0.0,UKBB_AG$DIAB_SDX_INS.0.0)
#no new information from insulin medication

#path 1: any diabetes self-report
# UKBB_AG$'DB_P1.0.0' <- ifelse(UKBB_AG$'DIAB_SDX.0.0'=="Yes"
#                               ,1
#                               ,0)

#path 2: current non-metformin oral diabetes medication (not sure how to lookup)
#identified coding from Eastwood diabetes paper
#first set of medicines are sulfonylureas (amaryl-tolbutamide)
#starlix - Meglitinides
#glitazones - pioglitazone etc.
#                               ,ifelse(UKBB_AG$'TRTMEDCODE.0.0' %in% c('amaryl 1mg tablet','diabinese 100mg tablet','gliclazide','glipizide','glipizide product','tolazamide','chlorpropamide',
#                                                                   'glimepiride','repaglinide','nateglinide','tolbutamide', 
#                                                                   'starlix 60mg tablet','pioglitazone','rosiglitazone')
#                                      ,1
#                                      ,0)
#                               ,0)

#path 3: age at dx greater than 36 (this whole column is null)
# UKBB_AG$'DIAB_AGE36.0.0' <- UKBB_AG$'DIAB_AGE.0.0' > 36
# UKBB_AG$'DB_P3.0.0' <- ifelse(UKBB_AG$'DB_P1.0.0'==1
#                             ,UKBB_AG$'DIAB_AGE36.0.0'
#                             ,UKBB_AG$'DB_P1.0.0')    
                              
                              
#path 4: exclude individuals with type 1 diabetes
#type 1 diabetics include, -9=exclude (t1d), 1=t2d, 0=non-diabetics
# UKBB_AG$'DB_P4.0.0' <- ifelse(UKBB_AG$'DB_P1.0.0'==1 & UKBB_AG$'DB_P3.0.0'==0
#                               ,ifelse(UKBB_AG$'TRTMEDCODE.0.0'=='insulin product' | UKBB_AG$'DIAB_SDX_INS.0.0'=='Yes',-9,1)
#                               ,UKBB_AG$'DB_P3.0.0')

#classify diabetes
# describe(UKBB_AG$'DB_P1.0.0')
# describe(UKBB_AG$'DB_P2.0.0')
# describe(UKBB_AG$'DB_P3.0.0')
# describe(UKBB_AG$'DB_P4.0.0')

#Restriction 4 : Remove T1D and Those with Missing Diabetes Status
# UKBB_AG <- UKBB_AG %>% filter(DB_P4.0.0 %in% c(0,1))
# paste0('Remove Type 1 Diabetics and Missing Diabetes')
# dim(UKBB_AG)

In [ ]:
#Export List of Individuals with Principal Components

#Third Restriction: EGFR or ALBUMINERIA value (valid)
UKBB_AG2 <- UKBB_AG %>% filter(!is.na(EGFR.0.0) & !is.na(ALBUMINERIA.0.0))
paste0('Kidney Disease (Valid Values), AND')
dim(UKBB_AG2)
UKBB_AG <- UKBB_AG %>% filter(!is.na(EGFR.0.0) | !is.na(ALBUMINERIA.0.0))
paste0('Kidney Disease (Valid Values), OR')
dim(UKBB_AG)

#Fourth Restriction: Individuals with AKI
UKBB_AG2 <- read.delim("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/AG_VARS2.txt", header = TRUE, na.strings=c("",".","NA"))
UKBB_AG2 <- UKBB_AG2 %>% select(f.eid, AKISOURCE.0.0)
#remove people with AKI within one year
UKBB_AG <- merge(UKBB_AG,UKBB_AG2,by="f.eid",all.x=TRUE) #left outer join
UKBB_AG <- UKBB_AG %>% filter(is.na(UKBB_AG$'AKISOURCE.0.0'))
dim(UKBB_AG)
#Prior restrictions
UKBB_AG <- UKBB_AG %>% filter(T2D.0.0 %in% c(0,1)) #no T1D diabetes
dim(UKBB_AG)

################EXPORT LIST################
PCLIST <- UKBB_AG %>% select(f.eid,T2D.0.0)
dim(PCLIST)
#447,482
write.table(PCLIST, file="~/jupyter/PCag_UKBB_03NOV20.txt", quote=FALSE, sep="\t", row.names=FALSE)

#Fifth Restriction: After exporting the list, reimport
#Limiting it to unrelated individuals
#names(PCAunrelated)
UKBB_AG <- merge(UKBB_AG,PCAunrelated,by.x="f.eid",by.y="FID_Salem",all.x=FALSE) #left outer join
dim(UKBB_AG)

In [ ]:
##########COVARIATES##########
NEWVAR <- read.delim("/data/nrnb03/dbGaP/users/e6richar/jupyter/UKB_Pheno/ALEXIS/AG_VARS2.txt", header = TRUE, na.strings=c("",".","NA"))
UKBB_AG2 <- merge(UKBB_AG,NEWVAR, by="f.eid", all.x=TRUE)
dim(UKBB_AG2)

#Education - 1997 International Standard Classification of Education (ISCED)
#of the United Nations Educational, Scientific and Cultural Organization
table(UKBB_AG2$'EDUCATION.0.0')
#prefer not to answer don't matter
UKBB_AG2$'EDUYEARS' <- ifelse(UKBB_AG2$'EDUCATION.0.0'=='College or University degree'
                                  ,20 #ISCED 5
                                  ,ifelse(UKBB_AG2$'EDUCATION.0.0'=='A levels/AS levels or equivalent'
                                          ,13 #ISCED 3
                                          ,ifelse(UKBB_AG2$'EDUCATION.0.0' %in% c('O levels/GCSEs or equivalent',
                                                                                  'CSEs or equivalent')
                                                  ,10 #2
                                                  ,ifelse(UKBB_AG2$'EDUCATION.0.0'=='Other professional qualifications eg: nursing, teaching'
                                                          ,15 #4
                                                          ,ifelse(UKBB_AG2$'EDUCATION.0.0'=='NVQ or HND or HNC or equivalent'
                                                              ,19 #5, diff college
                                                              ,ifelse(UKBB_AG2$'EDUCATION.0.0'=='Other professional qualifications eg: nursing, teaching'
                                                                  ,15 #4
                                                                  ,ifelse(UKBB_AG2$'EDUCATION.0.0'=='None of the above'
                                                                         ,7, #1
                                                                         NA))))))) #prefer not to answer group with NA
                                                  
summary(UKBB_AG2$'EDUYEARS')
#Lifestyle Score

#Smoking status - Current, Previous, Never
#describe(UKBB_AG$'SMOKE.0.0')                                                          

#Address prefer not to answer
#UKBB_AG2$DUR_VIG_PHY.0.0 <- ifelse(UKBB_AG2$DUR_VIG_PHY.0.0 < 0,NA,UKBB_AG2$DUR_VIG_PHY.0.0)
#UKBB_AG2$DUR_MOD_PHY.0.0 <- ifelse(UKBB_AG2$DUR_MOD_PHY.0.0 < 0,NA,UKBB_AG2$DUR_MOD_PHY.0.0)
#UKBB_AG2$FREQ_WK_VIG_PHY.0.0 <- ifelse(UKBB_AG2$FREQ_WK_VIG_PHY.0.0 < 0,NA,UKBB_AG2$FREQ_WK_VIG_PHY.0.0)
#UKBB_AG2$FREQ_WK_MOD_PHY.0.0 <- ifelse(UKBB_AG2$FREQ_WK_MOD_PHY.0.0 < 0,NA,UKBB_AG2$FREQ_WK_MOD_PHY.0.0)

#Physical activity 
#Freq is less than 0, then the other variable will be 0 as well
UKBB_AG2$VIG_TOT <- ifelse(UKBB_AG2$FREQ_WK_VIG_PHY.0.0==0,0,
                           floor(UKBB_AG2$DUR_VIG_PHY.0.0*UKBB_AG2$FREQ_WK_VIG_PHY.0.0))
UKBB_AG2$MOD_TOT <- ifelse(UKBB_AG2$FREQ_WK_MOD_PHY.0.0==0,0,
                           floor(UKBB_AG2$DUR_MOD_PHY.0.0*UKBB_AG2$FREQ_WK_MOD_PHY.0.0))
UKBB_AG2$COMBO_TOT <- UKBB_AG2$VIG_TOT + UKBB_AG2$MOD_TOT
#ideal - guidelines from prior papers; poor < 10 minutes per week
UKBB_AG2$PA_SCORE <- ifelse(UKBB_AG2$VIG_TOT>=75 | UKBB_AG2$MOD_TOT>=150 | UKBB_AG2$COMBO_TOT >=150
                           ,"ideal"
                           , ifelse(UKBB_AG2$VIG_TOT>=10 | UKBB_AG2$MOD_TOT>=10 | UKBB_AG2$COMBO_TOT >=10
                                    ,"intermediate"
                                    , ifelse(UKBB_AG2$VIG_TOT < 10 & !is.na(UKBB_AG2$VIG_TOT) | 
                                             UKBB_AG2$MOD_TOT < 10 & !is.na(UKBB_AG2$MOD_TOT) | 
                                             UKBB_AG2$COMBO_TOT < 10 & !is.na(UKBB_AG2$COMBO_TOT)
                                             ,"poor",NA)))
histogram(UKBB_AG2$VIG_TOT)
describe(UKBB_AG2$VIG_TOT)
histogram(UKBB_AG2$MOD_TOT)
describe(UKBB_AG2$MOD_TOT)
describe(UKBB_AG2$PA_SCORE)
table(is.na(UKBB_AG2$VIG_TOT),UKBB_AG2$PA_SCORE)                                                  
                                                          
##########FLOW-CHART##########
#Flow Chart
#QA - Kidney Disease
#List name of variables used
print('ACR')
summary(UKBB_AG2$'ACR.0.0')
print('eGFR')
summary(UKBB_AG2$'EGFR.0.0')
print('ESKD')
summary(UKBB_AG2$'ESKD.0.0')
print('CKD')
summary(UKBB_AG2$'CKD.0.0')
print('DN')
summary(UKBB_AG2$'DN.0.0')
print('All')
summary(UKBB_AG2$'ALL.0.0')
print('Non-ESKD')
summary(UKBB_AG2$'NONESKD.0.0')
print('DNCKD')
summary(UKBB_AG2$'DNCKD.0.0')
print('CTRL_DNCKD')
summary(UKBB_AG2$'CTRL_DNCKD.0.0')

#All vs. Albumineria
# UKBB_AG2 %>%
#   group_by(UKBB_AG2$'ALL.0.0',UKBB_AG2$'ALBUMINERIA.0.0') %>%
#   tally() %>%
#   data.frame() 


In [ ]:
###DIETARY INTAKE###
#Description: Servings for each score, 1 = met intake requirement, 0 = did not meet intake requirement

#FRUIT - 3 servings (intake)
#1 piece of fresh per day = 1 serving, 5 pieces of dried fruit per day = 1 serving
UKBB_AG2$"DIET_FRUIT_DRY_SERV.0.0" <- UKBB_AG2$"DIET_FRUIT_DRY.0.0"/5
#prefer not to answer
UKBB_AG2$DIET_FRUIT_FRESH.0.0 <- ifelse(UKBB_AG2$DIET_FRUIT_FRESH.0.0 < 0, NA, UKBB_AG2$DIET_FRUIT_FRESH.0.0)
UKBB_AG2$DIET_FRUIT_DRY_SERV.0.0 <- ifelse(UKBB_AG2$DIET_FRUIT_DRY_SERV.0.0 < 0, NA, UKBB_AG2$DIET_FRUIT_DRY_SERV.0.0)
#total servings
UKBB_AG2$FRUIT_SERV.0.0 <- rowSums(UKBB_AG2[,c("DIET_FRUIT_FRESH.0.0","DIET_FRUIT_DRY_SERV.0.0")], na.rm=TRUE)

summary(UKBB_AG2$FRUIT_SERV.0.0)
UKBB_AG2$FRUIT_SERV2.0.0 <- ifelse(UKBB_AG2$FRUIT_SERV.0.0 >= 3,1,0) 
#3 servings per day

#VEGETABLE - 3 tablespoons = 1 serving
#prefer not to answer
UKBB_AG2$DIET_SALAD_RAW.0.0<- ifelse(UKBB_AG2$DIET_SALAD_RAW.0.0 < 0, NA, UKBB_AG2$DIET_SALAD_RAW.0.0)
UKBB_AG2$DIET_COOK_VEG.0.0<- ifelse(UKBB_AG2$DIET_COOK_VEG.0.0< 0, NA, UKBB_AG2$DIET_COOK_VEG.0.0)
UKBB_AG2$VEG_SERV.0.0 <- rowSums(UKBB_AG2[,c("DIET_SALAD_RAW.0.0","DIET_COOK_VEG.0.0")], na.rm=TRUE)
summary(UKBB_AG2$VEG_SERV.0.0)
UKBB_AG2$VEG_SERV2.0.0 <- ifelse(UKBB_AG2$VEG_SERV.0.0/3 >= 3,1,ifelse(UKBB_AG2$VEG_SERV.0.0 < 0, NA, 0)) 
#3 heaped table spoons is one serving, 3 servings per day

#WHOLE GRAINS - 1 slice/bowl per day = 1 serving, 3 servings per day
#prefer not to answer
UKBB_AG2$DIET_BREAD_SLICESWK.0.0<- ifelse(UKBB_AG2$DIET_BREAD_SLICESWK.0.0 < 0, NA, UKBB_AG2$DIET_BREAD_SLICESWK.0.0)
UKBB_AG2$DIET_COOK_VEG.0.0<- ifelse(UKBB_AG2$DIET_COOK_VEG.0.0 < 0, NA, UKBB_AG2$DIET_COOK_VEG.0.0)
UKBB_AG2$GRAIN_SERV.0.0 <- rowSums(UKBB_AG2[,c("DIET_BREAD_SLICESWK.0.0","DIET_COOK_VEG.0.0")], na.rm=TRUE)
#identify whole grain or wholemeal
summary(UKBB_AG2$DIET_BREAD_TYPE.0.0)
#only count whole grain bread
UKBB_AG2$"DIET_BREAD_SLICESWK2.0.0" <- ifelse(UKBB_AG2$DIET_BREAD_TYPE.0.0=="Wholemeal or wholegrain",
                                             ifelse(UKBB_AG2$"DIET_BREAD_SLICESWK.0.0" >= 0,
                                                    UKBB_AG2$"DIET_BREAD_SLICESWK.0.0",NA),NA)
summary(UKBB_AG2$"DIET_BREAD_SLICESWK2.0.0")

#bowls, whole grain
UKBB_AG2$DIET_CBOWLSWK2.0.0<- ifelse(UKBB_AG2$DIET_CBOWLSWK.0.0 < 0, NA, UKBB_AG2$DIET_CBOWLSWK.0.0)
UKBB_AG2$"DIET_CBOWLSWK2.0.0" <- ifelse(UKBB_AG2$DIET_CTYPE.0.0 %in% c('Bran cereal (e.g. All Bran, Branflakes)'
                                                                          ,'Oat cereal (e.g. Ready Brek, porridge)'
                                                                          ,'Muesli'),
                                             ifelse(UKBB_AG2$"DIET_CBOWLSWK.0.0" >= 0,
                                                    UKBB_AG2$"DIET_CBOWLSWK.0.0",NA),NA)
summary(UKBB_AG2$"DIET_CBOWLSWK2.0.0")

UKBB_AG2$WGRAIN_SERV.0.0 <- rowSums(UKBB_AG2[,c("DIET_BREAD_SLICESWK2.0.0","DIET_CBOWLSWK2.0.0")], na.rm=TRUE)
UKBB_AG2$WGRAIN_SERV2.0.0 <- ifelse(UKBB_AG2$WGRAIN_SERV.0.0/3 >= 3,1,0) 
#3 servings per day
table(UKBB_AG2$WGRAIN_SERV2.0.0)

#Shellfish, Less than or equal to 2 servings 
UKBB_AG2$"DIET_OIL_FISH2.0.0" <- ifelse(UKBB_AG2$"DIET_OIL_FISH.0.0" %in% c('2-4 times a week',
                                                                            '5-6 times a week','Once or more daily')
                                        ,2,ifelse(UKBB_AG$"DIET_OIL_FISH.0.0" == 'Once a week',1,0))
UKBB_AG2$"DIET_NO_OIL_FISH2.0.0" <- ifelse(UKBB_AG2$"DIET_NO_OIL_FISH.0.0" %in% c('2-4 times a week',
                                                                            '5-6 times a week','Once or more daily')
                                        ,2,ifelse(UKBB_AG$"DIET_NO_OIL_FISH.0.0" == 'Once a week',1,0))

UKBB_AG2$FISH_SERV.0.0 <- rowSums(UKBB_AG2[,c("DIET_OIL_FISH2.0.0","DIET_NO_OIL_FISH2.0.0")], na.rm=TRUE)
summary(UKBB_AG2$FISH_SERV.0.0)
UKBB_AG2$FISH_SERV2.0.0 <- ifelse(UKBB_AG2$FISH_SERV.0.0 <= 2,1,0)

#Dairy, 2 servings per day
UKBB_AG2$"DIET_CHEESE_SERV.0.0" <- ifelse(UKBB_AG2$DIET_CHEESE.0.0 == 'Once or more daily',"yes","no")
UKBB_AG2$"DAIRY_SERV.0.0" <- ifelse(UKBB_AG2$DIET_CHEESE.0.0 == 'Once or more daily',"yes","no")
UKBB_AG2$"CHEESE_SERV.0.0" <- ifelse(UKBB_AG2$DIET_MILK.0.0 %in% c('Full cream','Never/rarely have milk',
                                                                   'Other type of milk','Semi-skimmed',
                                                                   'Skimmed','Soya'),"yes","no")
UKBB_AG2$"DIET_DAIRY_SERV.0.0" <- ifelse(UKBB_AG2$DAIRY_SERV.0.0 == 'yes' | 
                                         UKBB_AG2$CHEESE_SERV.0.0 == 'yes',1,0)

#Vegetable Oils
UKBB_AG2$"DIET_SPREAD_SERV.0.0" <- ifelse(UKBB_AG2$DIET_SPREAD.0.0 == 'Flora Pro-Active/Benecol',"yes","no")
UKBB_AG2$"DAIRY_SPREADT_SERV.0.0" <- ifelse(UKBB_AG2$DIET_SPREAD_TYPE.0.0 %in% c('Flora Pro-Active or Benecol'
                                                                          ,'Soft (tub) margarine'
                                                                          ,'Olive oil based spread (eg: Bertolli)'
                                                                          ,'Polyunsaturated/sunflower oil based spread (eg: Flora)'
                                                                          ,'Other low or reduced fat spread'),1,0)
#two spreads/day but only counts with two slices of bread
UKBB_AG2$"DAIRY_SPREAD2_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_SPREAD_SERV.0.0" == 'yes' & 
                                            UKBB_AG2$"DAIRY_SPREADT_SERV.0.0" == 'yes' & "DIET_BREAD_SLICESWK.0.0" >=2
                                           ,1,0)


#Refined Grains, less than or equal to 2 servings per day
UKBB_AG2$"DIET_REBREAD_SLICESWK2.0.0" <- ifelse(UKBB_AG2$DIET_BREAD_TYPE.0.0 %in% c('Brown','White','Other type of bread'),
                                             ifelse(UKBB_AG2$"DIET_BREAD_SLICESWK.0.0" >= 0,
                                                    UKBB_AG2$"DIET_BREAD_SLICESWK.0.0",NA),NA)
UKBB_AG2$"DIET_REFCBOWLSWK2.0.0" <- ifelse(UKBB_AG2$DIET_CTYPE.0.0 %in% c('Biscuit cereal (e.g. Weetabix)',
                                                                          'Other (e.g. Cornflakes, Frosties)') ,
                                             ifelse(UKBB_AG2$"DIET_CBOWLSWK.0.0" >= 0,
                                                    UKBB_AG2$"DIET_CBOWLSWK.0.0",NA),NA)
UKBB_AG2$"DIET_REFINE.0.0" <- rowSums(UKBB_AG2[,c("DIET_REBREAD_SLICESWK2.0.0","DIET_REFCBOWLSWK2.0.0")], na.rm=TRUE)
UKBB_AG2$"DIET_REFINE2.0.0" <- ifelse("DIET_REFINE.0.0" <= 2,1,0) 

#Processed Meats

#only code for consumption under coding
UKBB_AG2$"DIET_PROC_MEAT_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_PROC_MEAT.0.0" == "Once a week",
                                             1,
                                             ifelse(UKBB_AG2$"DIET_PROC_MEAT.0.0" == "Never",
                                             0,
                                             NA))
UKBB_AG2$"DIET_RECAGE_MEAT_SERV.0.0" <- ifelse("DIET_RECAGE_MEAT_SERV.0.0" == 0, 0, NA)
UKBB_AG2$"DIET_PROC_MEAT_SERV.0.0" <- rowSums(UKBB_AG2[,c("DIET_PROC_MEAT_SERV.0.0","DIET_RECAGE_MEAT_SERV.0.0")], na.rm=TRUE)

#less than one serving per week
UKBB_AG2$"DIET_PROC_MEAT_SERV2.0.0" <- ifelse(UKBB_AG2$"DIET_PROC_MEAT_SERV.0.0" <=1, #check servings less than 1
                                              #if ate more than a serving in a week, then automatically don't satisfy intake (redudant check)
                                              ifelse(UKBB_AG2$"DIET_PROC_MEAT.0.0" %in% c('2-4 times a week','5-6 times a week',
                                              'Less than once a week','Once or more daily','Prefer not to answer'),
                                              0,1),1) #no intake, intake, no intake
                                              
#Unprocessed Meats                              
UKBB_AG2$"DIET_POULTRY_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_POULTRY.0.0" == 'Once a week'
                                           ,1,ifelse(UKBB_AG2$"DIET_POULTRY.0.0" == "Never"
                                                ,0,ifelse(UKBB_AG2$"DIET_POULTRY.0.0" %in% c('2-4 times a week','5-6 times a week',
                                                          'Less than once a week','Once or more daily'),3,NA)))
UKBB_AG2$"DIET_BEEF_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_BEEF.0.0" == 'Once a week',
                                             1,
                                             ifelse(UKBB_AG2$"DIET_BEEF.0.0" == "Never",
                                             0,ifelse(UKBB_AG2$"DIET_BEEF.0.0" %in% c('2-4 times a week','5-6 times a week',
                                                          'Less than once a week','Once or more daily'),3,NA)))      
UKBB_AG2$"DIET_LAMB_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_LAMB.0.0" == 'Once a week',
                                             1,
                                             ifelse(UKBB_AG2$"DIET_LAMB.0.0" == "Never",
                                             0,ifelse(UKBB_AG2$"DIET_LAMB.0.0" %in% c('2-4 times a week','5-6 times a week',
                                                          'Less than once a week','Once or more daily'),3,NA)))                                              
UKBB_AG2$"DIET_PORK_SERV.0.0" <- ifelse(UKBB_AG2$"DIET_PORK.0.0" == 'Once a week',
                                             1,
                                             ifelse(UKBB_AG2$"DIET_PORK.0.0" == "Never",
                                             0,ifelse(UKBB_AG2$"DIET_PORK.0.0" %in% c('2-4 times a week','5-6 times a week',
                                                          'Less than once a week','Once or more daily'),3,NA)))
UKBB_AG2$"DIET_UNPROC_MEAT_SERV.0.0" <- ifelse("DIET_RECAGE_MEAT_SERV.0.0" == 0, 0, NA)
UKBB_AG2$"DIET_UNPROC.0.0" <- rowSums(UKBB_AG2[,c("DIET_POULTRY_SERV.0.0","DIET_BEEF_SERV.0.0",
                                                  "DIET_LAMB_SERV.0.0","DIET_PORK_SERV.0.0",
                                                   "DIET_UNPROC_MEAT_SERV.0.0")], na.rm=TRUE)
#less than or equal 2 servings per week
#first defined using counts                                              
UKBB_AG2$"DIET_UNPROC2.0.0" <- ifelse(UKBB_AG2$"DIET_REFINE.0.0" <= 2,1,0) #intake, no intake
                                              
#Sugar-sweetened beverages
UKBB_AG2$DIET_NEVER_SERV.0.0 <- ifelse(UKBB_AG2$DIET_NEVER.0.0 == 'Sugar or foods/drinks containing sugar', 1,
                                      ifelse(UKBB_AG2$DIET_NEVER.0.0 == 'I eat all of the above' 
                                             | UKBB_AG2$DIET_NEVER.0.0 == 'Prefer not to answer',0, NA))
                                        
                                        
#Ideal diet vs. poor diet - at least half of the components -> ideal (5/10)
UKBB_AG2$"IDEAL_DIET.0.0" <- rowSums(UKBB_AG2[,c("FRUIT_SERV2.0.0","VEG_SERV2.0.0","WGRAIN_SERV2.0.0",
                                                 "FISH_SERV2.0.0","DIET_DAIRY_SERV.0.0","DAIRY_SPREAD2_SERV.0.0","DIET_REFINE2.0.0",
                                                 "DIET_PROC_MEAT_SERV2.0.0","DIET_UNPROC2.0.0",
                                                 "DIET_NEVER_SERV.0.0")], na.rm=TRUE) 

UKBB_AG2$"IDEAL_DIET2.0.0" <- ifelse(UKBB_AG2$"IDEAL_DIET.0.0" >=5,"ideal","not ideal") %>% factor()
describe(UKBB_AG2$"IDEAL_DIET.0.0")
describe(UKBB_AG2$"IDEAL_DIET2.0.0")

In [ ]:
#Covariates

#Lifestyle Overall Score
UKBB_AG2$DIETCOUNT <- ifelse(UKBB_AG2$IDEAL_DIET2.0.0 =="ideal",1,0)
UKBB_AG2$PACOUNT <- ifelse(UKBB_AG2$PA_SCORE=="ideal",1,0)
UKBB_AG2$SMKCOUNT <- ifelse(UKBB_AG2$'SMOKE.0.0'=="Never",1,
                            ifelse(UKBB_AG2$SMOKE.0.0=="Prefer not to answer",NA,0))
UKBB_AG2$LIFESCORE <- rowSums(UKBB_AG2[,c("DIETCOUNT","PACOUNT","SMKCOUNT")],na.rm=TRUE) 
UKBB_AG2$LIFESCORE <- ifelse(UKBB_AG2$LIFESCORE > 2,2,UKBB_AG2$LIFESCORE) 
#Missing - replace with missing if smoke is prefer not to answer, or physical activity is missing
table(UKBB_AG2$LIFESCORE, useNA = "always")
UKBB_AG2$LIFESCORE <- ifelse(UKBB_AG2$SMOKE.0.0=="Prefer not to answer" | is.na(UKBB_AG2$PA_SCORE),NA,UKBB_AG2$LIFESCORE) %>% factor()
                                                    
table(UKBB_AG2$LIFESCORE, useNA = "always")

#Age
UKBB_AG2$'AGE.0.0' <- as.numeric(substr(UKBB_AG2$'VISDATE.0.0',1,4)) - as.numeric(UKBB_AG2$'YOB.0.0')

#Waist Hip Ratio
UKBB_AG2$'WHR.0.0' <- ifelse(!is.na(UKBB_AG2$'WAIST.0.0') & !is.na(UKBB_AG2$'HIP.0.0'),UKBB_AG2$'WAIST.0.0'/UKBB_AG2$'HIP.0.0',NA)
describe(UKBB_AG2$'WAIST.0.0')
describe(UKBB_AG2$'HIP.0.0')
describe(UKBB_AG2$'WHR.0.0')

In [ ]:
#Hypertension 

#Definition 
#Any antihypertension medications - see spreadsheet
hypermed <- read.delim("~/jupyter/HYPERMED.txt",header=TRUE,na.strings=c("",".","NA"),
                       fileEncoding="UCS-2LE")
hypermed2 <- hypermed %>% pull(Medication)

#Hypertension Medication Use
MED <-select(MED, 1, TRTMEDCODE.0.0:TRTMEDCODE.0.47)
MED1 <- MED %>% gather(MEDNUM,MEDCODE,TRTMEDCODE.0.0:TRTMEDCODE.0.47) #new columns, transpose
#Definition based off of the codes (anti-hypertensive)
MED1$HYPER<-ifelse(MED1$MEDCODE %in% unlist(hypermed2),1,0) 
##STATIN USE##
## https://www.biorxiv.org/content/10.1101/2020.08.28.272765v1.full#T2
##The statins included were atorvastatin, fluvastatin, pravastatin, rosuvastatin, and simvastatin.
##order: simvastatin,atorvastatin,rosuvastatin,pravastatin,fluvastatin,zocor,zocor heart-pro 10mg tablet,
##lipitor 10mg tablet
MED1$STATIN<-ifelse(MED1$MEDCODE %in% c(1140861958,1141146234,1141192410,1140888648,
                                          1140888648,1140888594,1141192414,1140881748,
                                          1141200040,1141146138),1,0)

DBCMED0 <-select(DBCMED, 1, DBC_DRUGS_F.0.0:DBC_DRUGS_F.0.3, DBC_DRUGS_M.0.0:DBC_DRUGS_M.0.2)
DBCMED1 <- DBCMED0 %>% gather(DBCNUM,DBCMED0,DBC_DRUGS_F.0.0:DBC_DRUGS_M.0.2)%>% filter(!is.na(DBCMED0))
#Other
DBCMED1$BP_MF<-ifelse(DBCMED1$DBCMED=="Blood pressure medication",1,0) 
DBCMED1$STATIN_MF<-ifelse(DBCMED1$DBCMED=="Cholesterol lowering medication",1,0) 

#Medication functions
ECOLLAPSE1<- function(data, old_col) {
old_col <- enquo(old_col)
new_col <- paste0("sum_", quo_name(old_col))
data %>% group_by(f.eid) %>%
mutate(!!new_col := ifelse(all(is.na(!!old_col)), NA, sum(!!old_col, na.rm = TRUE)))%>%
ungroup()}

ECOLLAPSE2<- function(data, old_col, sum_col) {
old_col <- enquo(old_col)
sum_col <- enquo(sum_col)
new_col2 <- paste0("MED_", quo_name(old_col))
data %>% mutate (!!new_col2 := case_when (!!sum_col >=1~1, !!sum_col==0~0,))}
 
MED1 <-MED1 %>% ECOLLAPSE1(HYPER) %>% ECOLLAPSE2(HYPER, sum_HYPER) %>% 
       ECOLLAPSE1(STATIN) %>% ECOLLAPSE2(STATIN, sum_STATIN)
MED2<-MED1%>% filter(MEDNUM=="TRTMEDCODE.0.0")%>% select(f.eid, MED_HYPER,
                                                         MED_STATIN) #96,200 from dataset

DBCMED1<-DBCMED1%>% ECOLLAPSE1(BP_MF) %>% ECOLLAPSE2(BP_MF, sum_BP_MF) %>% 
         ECOLLAPSE1(STATIN_MF) %>% ECOLLAPSE2(STATIN_MF, sum_STATIN_MF)
DBCMED2<-DBCMED1%>% filter(DBCNUM=="DBC_DRUGS_F.0.0"|DBCNUM=="DBC_DRUGS_M.0.0")%>% select(f.eid,MED_BP_MF,STATIN_MF) #Touchscreen 

#Merge medication files
HYPER_MED<- merge(DBCMED2, MED2, by=c("f.eid"), all = TRUE)

#112,667 with anti-hypertensive medication
table(HYPER_MED$MED_BP_MF,HYPER_MED$MED_HYPER)

##############IC9 and IC10 Codes from Inpatient Data##############

#Separate columns of inpatient data 
UKBINPs <- select(UKBINP, 1,ICD10.0.0:ICD9.0.46)
UKBINPsD <- select(UKBINP, 1,ICD10DATE.0.0:ICD9DATE.0.46)

#Reshape in R from wide to long and join columns
UKBINPl <- UKBINPs %>% gather(ICD,CODE,ICD10.0.0:ICD9.0.46)
UKBINPlD <- UKBINPsD %>% gather(ICDDATE,CODEDATE,ICD10DATE.0.0:ICD9DATE.0.46)
UKBINPL <- cbind(UKBINPl,UKBINPlD)
UKBINPL <- UKBINPL[c(1,2,3,6)]
UKBINPL <- UKBINPL %>% filter(!is.na(CODE))

#Limit Inpatient codes to baseline + 1 year
BASEDATE<-UKBB_AG  %>% select(f.eid, VISDATE.0.0)
UKBINPLbase<-merge(UKBINPL, BASEDATE, by="f.eid", all=T)
#head(UKBINPLbase)
UKBINPLbase$CODEDATE2<-as.Date(UKBINPLbase$CODEDATE)
UKBINPLbase$VISDATE<-as.Date(UKBINPLbase$VISDATE.0.0)
UKBINPLbase$VISDATE_1Y<-as.Date(UKBINPLbase$VISDATE.0.0)+365.25
UKBINPL1Y<-UKBINPLbase  %>% filter(CODEDATE2<=VISDATE_1Y)
#head(UKBINPL1Y)

#Hypertensive Disease
HYPCODE <- read.delim("~/jupyter/ICD910.txt",header=TRUE,na.strings=c("",".","NA"))
UKBINPL1Y$HYP<-ifelse(UKBINPL1Y$CODE %in% unlist(HYPCODE),1,0)#ICD
#Pre-diabetic
UKBINPL1Y$PREDB<-ifelse(UKBINPL1Y$CODE %in% c("790.29","R73"),1,0)#ICD R73.03

#Functions - generate new column name for ICD9/10
ECOLLAPSE1<- function(data, old_col) {
old_col <- enquo(old_col) #put quotes around it
new_col <- paste0("sum_", quo_name(old_col)) #short name with label
data %>% group_by(f.eid) %>%
mutate(!!new_col := ifelse(all(is.na(!!old_col)), NA, sum(!!old_col, na.rm = TRUE)))%>%
ungroup()}
ECOLLAPSE2<- function(data, old_col, sum_col) {
old_col <- enquo(old_col)
sum_col <- enquo(sum_col)
new_col2 <- paste0("INP_", quo_name(old_col)) #INP_HYP (medication field generated)
data %>% mutate (!!new_col2 := case_when (!!sum_col >=1~1, !!sum_col==0~0,))}

#add new hyp/predb icd9 var
#hypertension
UKBINPL1Y<-UKBINPL1Y%>%
ECOLLAPSE1(HYP) %>% ECOLLAPSE2(HYP, sum_HYP)
#diabetes
UKBINPL1Y<-UKBINPL1Y%>%
ECOLLAPSE1(PREDB) %>% ECOLLAPSE2(PREDB, sum_HYP)
##############Self-Report Illness (SDXILL)##############
UKBSDXILL<- select(SDXILL , 1,ILLNESS.0.0:ILLNESS.0.33)
UKBSDXILLL <- UKBSDXILL %>% gather(ILLNUM,ILLCODE,ILLNESS.0.0:ILLNESS.0.33)
UKBSDXILLL<-UKBSDXILLL %>% arrange(f.eid)
UKBSDXILLL$HYP <- case_when(UKBSDXILLL$ILLCODE==1065 | 
                               UKBSDXILLL$ILLCODE==1072 |
                               UKBSDXILLL$ILLCODE==1073 ~1,TRUE ~ 0) #Hypertension
table(UKBSDXILLL$HYP)

ECOLLAPSE1<- function(data, old_col) {
old_col <- enquo(old_col)
new_col <- paste0("sum_", quo_name(old_col))
data %>% group_by(f.eid) %>%
mutate(!!new_col := ifelse(all(is.na(!!old_col)), NA, sum(!!old_col, na.rm = TRUE)))%>%
ungroup()}
ECOLLAPSE2<- function(data, old_col, sum_col) {
old_col <- enquo(old_col)
sum_col <- enquo(sum_col)
new_col2 <- paste0("ILL_", quo_name(old_col))
data %>% mutate (!!new_col2 := case_when (!!sum_col >=1~1, !!sum_col==0~0,))}
 
UKBSDXILLL<-UKBSDXILLL %>% ECOLLAPSE1(HYP) %>% ECOLLAPSE2(HYP, sum_HYP)
UKBILL<-UKBSDXILLL%>% filter(ILLNUM=="ILLNESS.0.0")%>% select(f.eid, ILL_HYP) 

#Hypertension definition - dx, medication, self-report
HYPER_MED$f.eid <- as.integer(HYPER_MED$f.eid)
#only one dataframe needs to be deduplicated
UKBINPL1Y2 <- UKBINPL1Y %>% select(f.eid,INP_HYP,INP_PREDB) #ICD9/ICD10 dataset (select hyp,pred)
UKBINPL1Y2 <- distinct(UKBINPL1Y2)

#later merged to the full dataset
UKHYP <-full_join(UKBINPL1Y2,HYPER_MED) %>% full_join(., UKBILL) #includes all rows
dim(UKHYP) 

In [ ]:
##Medical Variables (Hypertension,Statin Use,Pre-Diabetes)

##Combine evidence from hypertension##
UKBB_AG3 <- merge(UKBB_AG2,UKHYP , by=c("f.eid"), all.x = FALSE)
dim(UKBB_AG3)

#Step 1 Identify those w/ self-reported hypertension or with medication
#SBP > 140 mmHg, dBP > 90 mmHg (two blood pressure fields for each)
UKBB_AG3<-UKBB_AG3 %>% mutate(HYP_POS1=case_when(INP_HYP==1|MED_BP_MF==1|MED_HYPER==1|ILL_HYP==1|
                             DBP.0.0 > 90|DBPm.0.0 > 90|SBP.0.0 > 140|SBPm.0.0 > 140~1,TRUE~0)) %>%
                       mutate(HYP_BP=case_when(DBP.0.0 > 90|DBPm.0.0 > 90|SBP.0.0 > 140|SBPm.0.0 > 140~1,TRUE~0))
UKBB_AG3$HYP_POS1 <- factor(UKBB_AG3$HYP_POS1,levels=c(1,0),labels=c("Hypertension","No Hypertension"))
table(UKBB_AG3$INP_HYP,UKBB_AG3$HYP_POS1)
table(UKBB_AG3$MED_BP_MF,UKBB_AG3$HYP_POS1)
table(UKBB_AG3$MED_HYPER,UKBB_AG3$HYP_POS1)
table(UKBB_AG3$ILL_HYP,UKBB_AG3$HYP_POS1)
table(UKBB_AG3$HYP_BP,UKBB_AG3$HYP_POS1)
table(UKBB_AG3$HYP_POS1)

##Statin Use
UKBB_AG3<-UKBB_AG3 %>% mutate(STATIN=case_when(STATIN_MF==1|MED_STATIN==1~1,TRUE~0)) 
UKBB_AG3$STATIN <- factor(UKBB_AG3$STATIN,levels=c(1,0),labels=c("Statin Use","No Statin Use"))
table(UKBB_AG3$STATIN)

##Pre-Diabetes (icd9, 5.7% to 6.4% AIC to 38.8 to 46.4 mmol/mol)
UKBB_AG3 <-UKBB_AG3 %>% mutate(PREDB_POS1=case_when(INP_PREDB==1|(HBA1C.0.0 > 38.8 & HBA1C.0.0 < 46.4) ~1,TRUE~0))
UKBB_AG3$PREDB_POS1 <- factor(UKBB_AG3$INP_PREDB,levels=c(1,0),labels=c("Pre-Diabetes","No Pre-Diabetes"))
table(UKBB_AG3$HYP_POS1)

In [ ]:
#PGScut-point fx
#note, unwrap quosure similar to name w/ sym object
cutpt <- function(data,var) {
    enquo_var <- enquo(var)
    decvar <- sym(paste0(rlang::quo_text(enquo_var),"d2")) #decile
    decvar2 <- sym(paste0(rlang::quo_text(enquo_var),"d3")) #10th and 90th per.
    qvar <- sym(paste0(rlang::quo_text(enquo_var),"q")) #convert char to name, quartile
    tvar <- sym(paste0(rlang::quo_text(enquo_var),"t")) #tertile
    fvar <- sym(paste0(rlang::quo_text(enquo_var),"f3")) #final ver
    
    #decile cutpoint
    data <- data %>% dplyr::mutate((!!decvar) := cut(data[,quo_text(enquo_var)], quantile(data[,quo_text(enquo_var)], probs=c(0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1),na.rm=T),
                                labels=c(1,2,3,4,5,6,7,8,9,10),include.lowest=TRUE))
    
    #10th and 90th percentile - keep for dependecies 
    #data <- data %>% dplyr::mutate((!!decvar2) := case_when((!!decvar) %in% c(2,3,4,5,6,7,8,9)~2,
    #                                                         (!!decvar)==1~1,
    #                                                         (!!decvar)==10~3)) %>% factor(labels = c("low risk",
                                                                                                       # "medium risk",
                                                                                                       # "high risk"))
    #tertile cutpoint
    data <- data %>% dplyr::mutate((!!tvar) := cut(data[,quo_text(enquo_var)], quantile(data[,quo_text(enquo_var)], probs=c(0:3/3),na.rm=T),
                                   labels=c(1,2,3),include.lowest=TRUE))
    #quartile cutpoint
    data <- data %>% dplyr::mutate((!!qvar) := cut(data[,quo_text(enquo_var)], quantile(data[,quo_text(enquo_var)],probs=c(0:4/4),na.rm=T),
                                    labels=c(1,2,3,4),include.lowest=TRUE))
    #final cutpoint
    data <- data %>% dplyr::mutate((!!fvar) := cut(data[,quo_text(enquo_var)], quantile(data[,quo_text(enquo_var)], probs=c(0,.10,.65,1.00),na.rm=T),
                                   labels=c(1,2,3),include.lowest=TRUE) %>% factor(labels = c("low risk",
                                    "medium risk","high risk")))
                                   
    return(data)

}

In [ ]:
#Polygenic Risk Score Section

#Raw and Weighted Scores were formed for each group.
#All of these scores are calculated in the overall population - quantiles in the overall population
#Only the weighted scores and T2D are calculated in the diabetes stratum.

#Rename the dataset again
UKBB_AG2 <- UKBB_AG3
#rm(UKBB_AG3)

####OVERALL POPULATIION PGS SCORE########
#Diabetes Status - reformat for table
dim(UKBB_AG2)
UKBB_AG2$'DB.0.0' <- UKBB_AG2$'T2D.0.0' %>% factor(levels=c(0,1),labels = c("non-diabetic","type 2 diabetic"))
table(UKBB_AG2$'DB.0.0')

#PGS - Insulin Resistance (Lipid Mediated - Raw)
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_LIR)
#PGS - Insulin Resistance (Other GWAS, no clusters)
#Raw and Weighted GRS_RAW_IR
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_IR)
#PGS - Insulin Resistance (full Lotta, 53 snps)
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_IR53)
#T2D Diabetes (126/128 snp)
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_T2DIR)
#Langour 2019 - L5E8
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_L5E8IR)
#Langour 2019 - L1E5
UKBB_AG2 <- cutpt(UKBB_AG2,GRS_WT_L1E5IR)


In [ ]:
#check decile and new category
#names(UKBB_AG2)


In [ ]:
#### Table 1 ####

####Biomarkers####
####Conversion to mg/dL units from mmol/L####
UKBB_AG2$LDL.0.0mg <- UKBB_AG2$LDL.0.0*38.67
UKBB_AG2$HDL.0.0mg <- UKBB_AG2$HDL.0.0*38.67
UKBB_AG2$TRIG.0.0mg <- UKBB_AG2$TRIG.0.0*88.57
summary(UKBB_AG2$LDL.0.0mg)
summary(UKBB_AG2$HDL.0.0mg)
summary(UKBB_AG2$TRIG.0.0mg)
summary(UKBB_AG2$SEX.0.0)
####Lipid Categories####
#normolipidemia
UKBB_AG2 <- UKBB_AG2 %>%
mutate(LPD=case_when(SEX.0.0 == 'Female' & HDL.0.0mg > 50 & LDL.0.0mg <160 & TRIG.0.0mg < 150 ~ 1,
                          SEX.0.0 == 'Male' & HDL.0.0mg > 40 & LDL.0.0mg <160 & TRIG.0.0mg < 150 ~ 1,
                     LDL.0.0mg >=160 & TRIG.0.0mg >=150 ~ 2, #combined hyperlipid
                     LDL.0.0mg >=160 & TRIG.0.0mg <150 ~ 3, #hypercholes
                     SEX.0.0 == 'Female' & HDL.0.0mg <= 50 & LDL.0.0mg <160 & TRIG.0.0mg >=150 ~ 4, #dys
                     SEX.0.0 == 'Male' & HDL.0.0mg <= 40 & LDL.0.0mg <160 & TRIG.0.0mg >=150 ~ 4,
                     SEX.0.0 == 'Female' & HDL.0.0mg <= 50 & LDL.0.0mg <160 & TRIG.0.0mg <150 ~ 5, #hdl
                     SEX.0.0 == 'Male' & HDL.0.0mg <= 40 & LDL.0.0mg <160 & TRIG.0.0mg <150 ~ 5,
                     SEX.0.0 == 'Female' & HDL.0.0mg > 50 & LDL.0.0mg <160 & TRIG.0.0mg >=150 ~ 6, #hyper
                     SEX.0.0 == 'Male' & HDL.0.0mg > 40 & LDL.0.0mg <160 & TRIG.0.0mg >=150 ~ 6))        
UKBB_AG2$LPD <- factor(UKBB_AG2$LPD,levels=c(1,2,3,4,5,6),labels=c("Normolipidemia","Combined hyperlipidemia","Combined hyperlipidemia",
                           "Dyslipidemia compatible with metabolic syndrome",
                           "Low HDL-c","Hypertriglyceridemia"))
table(UKBB_AG2$LPD)  

#### Continuous Variables ####
continuousVars <- c('AGE.0.0','SES_TDI.0.0','BMI.0.0','EDUYEARS','SBP.0.0','DBP.0.0',
                    'HDL.0.0mg','LDL.0.0mg','TRIG.0.0mg','VIG_TOT','MOD_TOT')
#### Categorical Variables ####
catVars<- c('SEX.0.0','TOTAL_INCOME.0.0','IDEAL_DIET2.0.0',
            'SMOKE.0.0','STATIN','LIFESCORE','PA_SCORE','LPD','PREDB_POS1','HYP_POS1')
labels <- list(variables=list('SEX.0.0'="Gender",'AGE.0.0'="Age",'EDUYEARS'="Education",'TOTAL_INCOME.0.0'="Total Income",
                              'SES_TDI.0.0'="Townsend Deprivation Index",'SMOKE.0.0'='Smoking Status',
                              'PA_SCORE'="Physical Activity",'VIG_TOT'='Vigorous PA',
                              'MOD_TOT'='Moderate PA','IDEAL_DIET2.0.0'="Ideal Diet", 'LIFESCORE'="Lifestyle Score",
                              'BMI.0.0'="Body Mass Index",'SBP.0.0'='Systolic BP','DBP.0.0'='Diastolic BP',
                              'STATIN'='Statin Use','PREDB_POS1'='Pre-Diabetes','HYP_POS1'='Hypertension',
                              'HDL.0.0mg'='HDL','LDL.0.0mg'='LDL','TRIG.0.0mg'='TRIG','LPD'='Dyslipidemia category'), 
               groups=list("","Overall"))

#strata, total sample to start
#non-diabetic (LIR)
#ndb_strata <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",],UKBB_AG2$'GRS_RAW_LIR2d2'))
#diabetic (LIR)
#db_strata <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",],UKBB_AG2$'GRS_RAW_LIR2d2'))
#non-diabetic (IR)
#ndb_strataIR2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",],UKBB_AG2$'GRS_RAW_IR2d2'))
#diabetic (IR)
#db_strataIR2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",],UKBB_AG2$'GRS_RAW_IR2d2'))
#non-diabetic (Weighted IR)
ndb_strataWIR2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",],UKBB_AG2$'GRS_WT_IRf3'))
#diabetic (Weighted IR)
db_strataWIR2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",],UKBB_AG2$'GRS_WT_IRf3'))
#langour v1
#ndb_strataL5E8I2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",],UKBB_AG2$'GRS_WT_L5E8IR2d2'))
#db_strataL5E8I2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",],UKBB_AG2$'GRS_WT_L5E8IR2d2'))
#langour v2
#ndb_strataL1E5I2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="non-diabetic",],UKBB_AG2$'GRS_WT_L1E5IR2d2'))
#db_strataL1E5I2 <- c(list(Total=UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",]),split(UKBB_AG2[UKBB_AG2$'DB.0.0'=="type 2 diabetic",],UKBB_AG2$'GRS_WT_L1E5IR2d2'))

my.render.cont <- function(x) {
  with(stats.apply.rounding(stats.default(x), digits=2), c("","Mean (SD)"=sprintf("%s (&plusmn; %s)", MEAN, SD)))
}
my.render.cat <- function(x) {
  c("", sapply(stats.default(x), function(y) with(y,sprintf("%d (%0.0f)", FREQ, PCT))))
}

#Lipid IR
# Table1a <- table1(ndb_strata,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
# Table1a
# Table1b <- table1(db_strata,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
# Table1b              
#filename <- paste0("/Users/alexisgarduno/Documents/2020-2021/Fall 2020/FPM259A/Analysis/Tables/Ta5ble1.html")
#write.table(Table1, file=filename,col.names=NA)

#IR GWAS
# Table2a <- table1(ndb_strataIR2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
# Table2a
# Table2b <- table1(db_strataIR2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
# Table2b
               
#IR GWAS (WEIGHTED)   
Table3a <- table1(ndb_strataWIR2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
Table3a
Table3b <- table1(db_strataWIR2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
Table3b
               
#Langour GWAS (1x10^-5)
#Table4a <- table1(ndb_strataL5E8I2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
#Table4a
#Table4b <- table1(db_strataL5E8I2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
#Table4b               

#Langour GWAS (5x10^-8)
#Table5a <- table1(ndb_strataL1E5I2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
#Table5a
#Table5b <- table1(db_strataL1E5I2,labels=labels,groupspan=c(1,3),render.continuous=my.render.cont,render.categorical=my.render.cat)
#Table5b   
               
#Overall Sample

In [ ]:
#names(UKBB_AG2)

In [ ]:
#Writing the final table (whole dataset)
write.table(UKBB_AG2,file="~/jupyter/UKBB_AG2_12Jan21.txt", quote=FALSE, sep="\t", row.names=FALSE)

###Exclude Hypertension###
#Finally change 
UKBB_AG2_no <- UKBB_AG2 %>% filter(HYP_POS1=="No Hypertension")
table(UKBB_AG2_no$HYP_POS1)
table(UKBB_AG2$HYP_POS1)

#Writing the final table (those without hypertension)
write.table(UKBB_AG2_no,file="~/jupyter/UKBB_AG2_nohyp_12Jan21.txt", quote=FALSE, sep="\t", row.names=FALSE)

###Exclude Pre-Diabetes###
UKBB_AG2_nopdb <- UKBB_AG2 %>% filter(PREDB_POS1=="No Pre-Diabetes")
table(UKBB_AG2_nopdb$PREDB_POS1)
write.table(UKBB_AG2_nopdb,file="~/jupyter/UKBB_AG2_nopDB_12Jan21.txt", quote=FALSE, sep="\t", row.names=FALSE)

###Exclude Hypertension and Pre-Diabetes##
UKBB_AG2_noboth <- UKBB_AG2 %>% filter(PREDB_POS1=="No Pre-Diabetes" & HYP_POS1=="No Hypertension")
table(UKBB_AG2_noboth$PREDB_POS1)
table(UKBB_AG2_noboth$HYP_POS1)
write.table(UKBB_AG2_noboth,file="~/jupyter/UKBB_AG2_noboth_12Jan21.txt", quote=FALSE, sep="\t", row.names=FALSE)